Prepare the environment

In [3]:
! pip install azure-ai-ml 
! pip install azure-identity 
! pip install keras
! pip install tensorflow
! pip install pandas
! pip install opencv-python
! pip install Pillow
! pip install fileupload

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
from azure.identity import DefaultAzureCredential
import io
import fileupload
from IPython.display import display

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<AML_WORKSPACE_NAME>"

    ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

#def handle_file_upload(change):
    # Get the uploaded file
#    uploaded_file = change['owner'].data[0]
    
    # Read the file contents
#    file_contents = uploaded_file['content']

# Create the file upload widget
#file_upload_widget = fileupload.FileUploadWidget()

# Attach the handle_file_upload function to the widget
#file_upload_widget.observe(handle_file_upload, names='data')

# Display the widget
#display(file_upload_widget)

We could not find config.json in: . or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.


FileUploadWidget(label='Browse', _dom_classes=('widget_item', 'btn-group'))

In [20]:
import os
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import glob
import cv2
from helper_functions import imgs_to_array, get_lcd
from PIL import Image

# Directory structure:
root = '../Test_case/'
data = root + 'test_data/'
results = root + 'results/'
if not os.path.exists(results):
    os.mkdir(results)
lcd_data = 'temp/'
if not os.path.exists(lcd_data):
    os.mkdir(lcd_data)

#Load CNN model trained on data pre-defined in the paper
model=load_model('../Dataset/best_model.h5')

# Preprocess image in into binarized single monitor lcd_data with a label file
fname = data+'image1.jpg'

preprocessed_img = get_lcd(fname)
w, h=preprocessed_img.shape
top =  preprocessed_img[0:int(h/2),0:w]
top = Image.fromarray(top)
top = top.resize((180,80))
top = np.array(top)
top = top.reshape((top.shape[0],top.shape[1],1))
top_array = np.asarray([top])
sys_pred = model.predict(top_array)
for i in range(top_array.shape[0]):
    pred_list_i = [np.argmax(pred[i]) for pred in sys_pred]
    predicted_sys = 100* pred_list_i[0] + 10 * pred_list_i[1] + 1* pred_list_i[2]
    if predicted_sys >= 1000:
        predicted_sys = predicted_sys-1000

bottom = preprocessed_img[int(h/2):h,0:w]
bottom = Image.fromarray(bottom)
bottom = bottom.resize((180,80))
bottom = np.array(bottom)
bottom = bottom.reshape((bottom.shape[0],bottom.shape[1],1))
bottom_array = np.asarray([bottom])
dia_pred = model.predict(bottom_array)
for i in range(bottom_array.shape[0]):
    pred_list_i = [np.argmax(pred[i]) for pred in dia_pred]
    predicted_dia = 100* pred_list_i[0] + 10 * pred_list_i[1] + 1* pred_list_i[2]
    if predicted_dia >= 1000:
        predicted_dia = predicted_dia-1000

print("Predicted systolic and diastolic values are: ",predicted_sys,predicted_dia)


../Test_case/test_data/image1.jpg
BP 76 132 311 256 79616 1.21484375 76764.0
1/1 [==============================] - 0s 38ms/step
Predicted systolic and diastolic values are:  105 72
